# 💬🌽 Semantic Kernel Agents V1 Time Capsule

This is going to change in Semantic Kernel 1.10, but it's been swimming around in my head since SK 1.0 to better understand. It walks through how to: 1/ give the agent an SK Plugin, 2/ use the OAI Assistants retrieval option with files, and 3/ use the code interpreter to create a PowerPoint presentation file.

## 📦 Get the right packages together

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.10.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.10.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.10.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.10.0-alpha"

Installed Packages Microsoft.SemanticKernel, 1.10.0 Microsoft.SemanticKernel.Agents.Core, 1.10.0-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.10.0-alpha Microsoft.SemanticKernel.Plugins.Core, 1.10.0-alpha

## Create some helper functions to keep track of resources

In [2]:
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.Extensions.Logging;
using System.ComponentModel;
using System.Net.Http;
using System.Net.Http.Headers;
using Microsoft.Extensions.DependencyInjection;

#pragma warning disable SKEXP0010, SKEXP0101, CS0103, SKEXP0110

using Kernel = Microsoft.SemanticKernel.Kernel;

// Kernel kernel;
static List<OpenAIAssistantAgent> _agents = [];
static string[] _fileIds = [];

// This sample must use OpenAI Assistants (not Azure OpenAI) because it uses the retrieval tool
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

model = "gpt-4-turbo";

ServiceProvider serviceProvider = new ServiceCollection()
    .AddOpenAIChatCompletion(model, apiKey)
    .BuildServiceProvider();

public static OpenAIAssistantAgent Track(OpenAIAssistantAgent agent)
{
    _agents.Add(agent);

    return agent;
}

public async Task<string> UploadAgentsRelatedFile(string filename)
{
    FileStream fileStream = new FileStream(filename, FileMode.Open);

    OpenAIFileService fileService = new(apiKey);

    OpenAIFileReference uploadFile =
        await fileService.UploadContentAsync(
            new BinaryContent(() => Task.FromResult((Stream)fileStream)),
            new OpenAIFileUploadExecutionSettings(filename, OpenAIFilePurpose.Assistants));  

    _fileIds = _fileIds.Append(uploadFile.Id).ToArray();
    return uploadFile.Id;
}

public async Task CleanupAgents()
{

    OpenAIFileService fileService = new(apiKey);

    await Task.WhenAll(_agents.Select(a => a.DeleteAsync()));
    await Task.WhenAll(_fileIds.Select(fileId => fileService.DeleteFileAsync(fileId)));

    _agents = new List<OpenAIAssistantAgent>();
    _fileIds = new string[] {};
}

## 🔌 Plugins can be handed over to any of your agents

In [3]:
public class TimeInformationPlugin
{
    [KernelFunction]
    [Description("Retrieves the current time in UTC.")]
    public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
}

var timePlugin = KernelPluginFactory.CreateFromType<TimeInformationPlugin>();


## 🐣 Hatch a few 🥸 agents and provide 📁 files and 🔌 plugins for grounding

In [4]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103, SKEXP0110

var fileId = await UploadAgentsRelatedFile("travelinfo.txt");

// You can create an OpenAI Assisstant with the retrieval tool enabled so it can find information in the uploaded file
OpenAIAssistantAgent retrievalAgent =
    await OpenAIAssistantAgent.CreateAsync(
        kernel: new(),
        config: new(apiKey),
        new()
        {
            Name = "Retrieval",
            Description = "Really good at finding travel information about people using travel docs",
            Instructions = "Respond succinctly with the information requested.",
            EnableRetrieval = true, // Enable retrieval
            ModelId = model,
            FileIds = [fileId] // Associate uploaded file
        });
Track(retrievalAgent);

ChatCompletionAgent timeAgent =
    new()
    {
        Instructions = "Respond succinctly with the information requested.",
        Name = "Timeman",
        Description = "Knows what time it is",
        Kernel = new(
            serviceProvider,
            new ([timePlugin])
        ),
        ExecutionSettings = new OpenAIPromptExecutionSettings() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions },
    };


async Task InvokeAgentAsync(AgentGroupChat agentChat, KernelAgent? agent)
{
    var messages = agent == null ? agentChat.InvokeAsync() : agentChat.InvokeAsync(agent);

    await foreach (var message in messages)
    {
        string content = message.Content;
        List<AnnotationContent> annotations = message.Items.OfType<AnnotationContent>().ToList();

        Console.WriteLine($"# {message.Role}: {content}");

        if (annotations.Count > 0)
        {
            OpenAIFileService fileService = new(apiKey);
            Console.WriteLine("\nAnnotation related files:");
            foreach (var annotation in annotations)
            {
                Console.WriteLine($"* {annotation.FileId}");
            }
        }
    }
    Console.WriteLine();
}

## 💬 Chat with the 🥸 agent of your choice 

When you run the code below, the notebook is asking a question at the very to of the screen. 👆 Look up at the top of your VS Code window!

![](assets/lookup.png)

Get it? When you run the code below you need to 👆 go to the top of this window to enter your input. I always forget that ...

> ⚠️ _Be sure to clean up your agent work by going to the cell just below this one_

In [5]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103, SKEXP0110

// Create a chat with the agents
AgentGroupChat chat = new(timeAgent, retrievalAgent);

try
{
    // sit in a loop and talk to the agent and when you enter /bye then exit
    string myq;

    do
    {
        myq = await InteractiveKernel.GetInputAsync("Your turn: ");

        // Add the user's message to the chat
        chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, myq));

        // In this chat we will always talk to the retrieval agent
        await InvokeAgentAsync(chat, retrievalAgent);
    } while (myq != "/bye");
}
finally
{
    Console.WriteLine("⚠️ Completed. Be sure to clean up your agents.");
}


# assistant: Sam's flight is scheduled to depart on Monday, 11 September 2023 at 10:05 AM from Seattle (SEA) to Durham (RDU) on flight AS 572 with Alaska Airlines.

# assistant: Goodbye! If you have any more questions in the future, feel free to ask. Have a great day!

⚠️ Completed. Be sure to clean up your agents.


## 🪚 Build a simple agent 👮 router

In [6]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103, SKEXP0110

KernelFunction selectionFunction =
    KernelFunctionFactory.CreateFromPrompt(
        $$$"""
        Provide _just_ the assistant's name that can answer the very last message from the user in the following chat:
        
        {{$history}}

        Choose only from these assistants:
        - Name: {{{retrievalAgent.Name}}}
          Description: {{{retrievalAgent.Description}}}
        - Name: {{{timeAgent.Name}}}
          Description: {{{timeAgent.Description}}}

        If you don't know, respond with {{{retrievalAgent.Name}}}
        """);

## 🧪 Test the agent 👮 router to see if it works

In [10]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103, SKEXP0110

AgentGroupChat routingChat =
    new(timeAgent, retrievalAgent)
    {
        ExecutionSettings =
            new()
            {
                // Here a KernelFunctionSelectionStrategy selects agents based on a prompt function.
                SelectionStrategy =
                    new KernelFunctionSelectionStrategy(selectionFunction, new(serviceProvider))
                    {
                        // Returns the entire result value as a string.
                        ResultParser = (result) => {
                            Console.WriteLine($"Selected agent: {result.GetValue<string>()}");
                            return result.GetValue<string>() ?? timeAgent.Name;
                        },
                        // The prompt variable name for the history argument.
                        HistoryVariableName = "history",
                    },
            }
    };

try
{
    // sit in a loop and talk to the agent and when you enter /bye then exit
    string myq;

    do {
        myq = await InteractiveKernel.GetInputAsync("Your turn: ");
        
        // Add the user's message to the chat
        routingChat.AddChatMessage(new ChatMessageContent(AuthorRole.User, myq));

        await InvokeAgentAsync(routingChat, null);
    } while (myq != "/bye");
}
finally
{
    Console.WriteLine("⚠️ Completed. Be sure to clean up your agents.");
}

# assistant: Hello! How can I assist you today?

Selected agent: Timeman
# assistant: The current UTC time is 22:50 (10:50 PM). How else can I help you today?

Selected agent: Retrieval
# assistant: Sam's flight is scheduled for Monday, 11th September 2023, departing from Seattle (SEA) at 10:05 AM and arriving at Durham (RDU) at 6:15 PM. The flight is AS 572 operated by Alaska Airlines.

Selected agent: Retrieval
# assistant: Today is the 18th of September 2023. Sam's flight was on the 11th of September 2023, which means it was 7 days ago.

Selected agent: Retrieval
# assistant: Goodbye! If you need any more help in the future, feel free to ask. Have a great day!

⚠️ Completed. Be sure to clean up your agents.


## 🧽 Clean up your 🥸 agents and any 📁 files you uploaded

Various items will stay inside your account unless you consciously purge them. Otherwise you'll have to manually delete them 🙀 ...

In [ ]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103
// print the current time in the current timezone
Console.WriteLine($"Current time: {DateTime.Now}");

Console.WriteLine("🧽 Cleaning up...");
await CleanupAgents();
Console.WriteLine("👋 Done. Bye!");

## ⛰️ Advanced: Let the agent run 🐍 Python code to write a file

In [11]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103, SKEXP0110

var fileId = await UploadAgentsRelatedFile("travelinfo.txt");

// Define the agent
OpenAIAssistantAgent codeInterpreterPlusRetrievalAgent =
    await OpenAIAssistantAgent.CreateAsync(
        kernel: new(),
        config: new(apiKey),
        new()
        {
            Name = "Crafter",
            Description = "Really good at crafting code, finding info about people, and creating powerpoints.",
            Instructions = "Write only code to solve the given problem without comment.",
            EnableCodeInterpreter = true, // Enable code-interpreter
            EnableRetrieval = true, // Enable retrieval
            ModelId = model,
            FileIds = [fileId] // Associate uploaded file
        });
Track(codeInterpreterPlusRetrievalAgent);


// Create a chat with the agents
AgentGroupChat codeInterpreterChat = new(codeInterpreterPlusRetrievalAgent);

async Task InvokeAgentAsyncSaveToFile(string question, string? outputFileName = null)
{
    codeInterpreterChat.AddChatMessage(new ChatMessageContent(AuthorRole.User, question));

    await foreach (var message in codeInterpreterChat.InvokeAsync(codeInterpreterPlusRetrievalAgent))
    {
        string content = message.Content;
        List<AnnotationContent> annotations = message.Items.OfType<AnnotationContent>().ToList();

        Console.WriteLine($"# {message.Role}: {content}");

        if (annotations.Count > 0)
        {
            OpenAIFileService fileService = new(apiKey);

            Console.WriteLine("\nAnnotation related files:");
            foreach (var annotation in annotations)
            {
                Console.WriteLine($"* {annotation.FileId}");
                if (outputFileName != null) {
                    // Download file if given a name
                    // Note that if there are multiple files then this will not work the way you want
                    using (var httpClient = new HttpClient())
                    {
                        httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", apiKey);
                        var response = await httpClient.GetAsync($"https://api.openai.com/v1/files/{annotation.FileId}/content");
                        var fileContent = await response.Content.ReadAsByteArrayAsync();

                        // Write file to local directory
                        Console.WriteLine($"Writing file to {outputFileName}");
                        await File.WriteAllBytesAsync(outputFileName, fileContent);
                        Console.WriteLine("File written.");
                    }
                }          
            }
        }
    }
    Console.WriteLine();
}

## 🚵 Run the 🥸 agent with 🧑‍💻 code interpreter

This took me about 50 seconds to complete. So be a little patient while crossing your fingers ...

In [ ]:
await InvokeAgentAsyncSaveToFile(
    "Create a powerpoint file that documents Sam's travel plans.", 
    "sam.pptx");
Console.WriteLine("👋 Done. Bye!");

## 🧽 Clean up your 🥸 agents and any 📁 files you uploaded

This is missing the file you created ... I'll let you figure out how to do that ;-).

In [ ]:
#pragma warning disable SKEXP0010, SKEXP0101, CS0103
// print the current time in the current timezone
Console.WriteLine($"Current time: {DateTime.Now}");

Console.WriteLine("🧽 Cleaning up...");
await CleanupAgents();
Console.WriteLine("👋 Done. Bye!");